# Bedrock with LangChain - Code Translation from one programming language to another

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to generate an email response to a customer who was not happy with the quality of customer service that they received from the customer support engineer. We will provide additional context to the model by providing the contents of the actual email that was received from the unhappy customer.

Because of additional context in the prompt, the text produced by the Amazon Titan Large language model in this notebook is of much better quality and relevance than the content produced earlier through zero-shot prompts.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework for developing applications powered by language models. The key aspects of this framework allow us to augment the Large Language Models by chaining together various components to create advanced use cases.

In this notebook we will use the Bedrock API provided by LangChain. The prompt used in this example creates a custom LangChain prompt template for adding context to the text generation request. 

**Note:** *This notebook can be run within or outside of AWS environment.*

#### Context
In the previous example `02_code_interpret_w_langchain.ipynb`, we explored how to use LangChain framework to communicate with Amazon Bedrock API. In this notebook we will try to add a bit more complexity with the help of `PromptTemplates` to leverage the LangChain framework for the similar use case. `PrompTemplates` allow you to create generic shells which can be populated with information later and get model outputs based on different scenarios.

As part of this notebook we will explore the use of Amazon Bedrock integration within LangChain framework and how it could be used to generate text with the help of `PromptTemplate`.

#### Pattern
We will simply provide the LangChain implementation of Amazon Bedrock API with an input consisting of a task, an instruction and an input for the model under the hood to generate an output without providing any additional example. The purpose here is to demonstrate how the powerful LLMs easily understand the task at hand and generate compelling outputs.

![](./images/bedrock-code-gen-langchain.png)

#### Use case
To demonstrate the generation capability of models in Amazon Bedrock, let's take the use case of email generation.

#### Persona
You are Bob a Customer Service Manager at AnyCompany and some of your customers are not happy with the customer service and are providing negative feedbacks on the service provided by customer support engineers. Now, you would like to respond to those customers humbly aplogizing for the poor service and regain trust. You need the help of an LLM to generate a bulk of emails for you which are human friendly and personalized to the customer's sentiment from previous email correspondence.

#### Implementation
To fulfill this use case, we will show you how to generate an email with a thank you note based on the customer's previous email. We will use the Amazon Titan Text Large model using the Amazon Bedrock LangChain integration. 


## Setup

Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

For more details on how the setup works and ⚠️ **whether you might need to make any changes**, refer to the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

In this notebook, we'll also install the [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) library which we'll use for counting the number of tokens in an input prompt.

In [1]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet langchain==0.0.249 "transformers>=4.24,<5"

Processing /Users/mundabra/dev/bedrock/amazon-bedrock-workshop/dependencies/awscli-1.29.21-py3-none-any.whl
Processing /Users/mundabra/dev/bedrock/amazon-bedrock-workshop/dependencies/boto3-1.28.21-py3-none-any.whl
Processing /Users/mundabra/dev/bedrock/amazon-bedrock-workshop/dependencies/botocore-1.31.21-py3-none-any.whl
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Using cached s3transfer-0.6.1-py3-none-any.whl (79 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-macosx_10_9_x86_64.whl (189 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached pyasn1-0.5.0-py2.py3-none-any.whl (83 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.16
  

In [3]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "fine-tuning-bedrock"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
  Using profile: fine-tuning-bedrock
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


## Invoke the Bedrock LLM Model

We'll begin with creating an instance of Bedrock class from llms. This expects a `model_id` which is the ARN of the model available in Amazon Bedrock. 

Optionally you can pass on a previously created boto3 client as well as some `model_kwargs` which can hold parameters such as `temperature`, `topP`, `maxTokenCount` or `stopSequences` (more on parameters can be explored in Amazon Bedrock console).

Available text generation models under Amazon Bedrock have the following IDs:

- amazon.titan-tg1-large
- ai21.j2-grande-instruct
- ai21.j2-jumbo-instruct
- anthropic.claude-instant-v1
- anthropic.claude-v1

Note that different models support different `model_kwargs`.

In [4]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )


## Create a LangChain custom prompt template

By creating a template for the prompt we can pass it different input variables to it on every run. This is useful when you have to generate content with different input variables that you may be fetching from a database.

Previously we hardcoded the prompt, it might be the case that you have multiple customers sending similar negative feedback and you now want to use each of those customer's emails and respond to them with an apology but you also want to keep the response a bit personalized. In the following cell we are exploring how you can create a `PromptTemplate` to achieve this pattern.

In [5]:
# Vehicle Fleet Management Code written in C++
sample_code = """
#include <iostream>
#include <string>
#include <vector>

class Vehicle {
protected:
    std::string registrationNumber;
    int milesTraveled;
    int lastMaintenanceMile;

public:
    Vehicle(std::string regNum) : registrationNumber(regNum), milesTraveled(0), lastMaintenanceMile(0) {}

    virtual void addMiles(int miles) {
        milesTraveled += miles;
    }

    virtual void performMaintenance() {
        lastMaintenanceMile = milesTraveled;
        std::cout << "Maintenance performed for vehicle: " << registrationNumber << std::endl;
    }

    virtual void checkMaintenanceDue() {
        if ((milesTraveled - lastMaintenanceMile) > 10000) {
            std::cout << "Vehicle: " << registrationNumber << " needs maintenance!" << std::endl;
        } else {
            std::cout << "No maintenance required for vehicle: " << registrationNumber << std::endl;
        }
    }

    virtual void displayDetails() = 0;

    ~Vehicle() {
        std::cout << "Destructor for Vehicle" << std::endl;
    }
};

class Truck : public Vehicle {
    int capacityInTons;

public:
    Truck(std::string regNum, int capacity) : Vehicle(regNum), capacityInTons(capacity) {}

    void displayDetails() override {
        std::cout << "Truck with Registration Number: " << registrationNumber << ", Capacity: " << capacityInTons << " tons." << std::endl;
    }
};

class Car : public Vehicle {
    std::string model;

public:
    Car(std::string regNum, std::string carModel) : Vehicle(regNum), model(carModel) {}

    void displayDetails() override {
        std::cout << "Car with Registration Number: " << registrationNumber << ", Model: " << model << "." << std::endl;
    }
};

int main() {
    std::vector<Vehicle*> fleet;

    fleet.push_back(new Truck("XYZ1234", 20));
    fleet.push_back(new Car("ABC9876", "Sedan"));

    for (auto vehicle : fleet) {
        vehicle->displayDetails();
        vehicle->addMiles(10500);
        vehicle->checkMaintenanceDue();
        vehicle->performMaintenance();
        vehicle->checkMaintenanceDue();
    }

    for (auto vehicle : fleet) {
        delete vehicle; 
    }

    return 0;
}
"""

In [6]:
from langchain import PromptTemplate

# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["code", "srcProgrammingLanguage", "targetProgrammingLanguage"], 
    template="""Human: You will be acting as an expert software developer in {srcProgrammingLanguage} and {targetProgrammingLanguage}. 
                You will tranlslate below code from {srcProgrammingLanguage} to {targetProgrammingLanguage} while following coding best practices.
                {code}
                Assistant: 
            """
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(code=sample_code, srcProgrammingLanguage="C++", targetProgrammingLanguage="Java")


### Code translation from C++ to Java

In [7]:
response = textgen_llm(prompt)

target_code = response[response.index('\n')+1:]

print_ww(target_code)


```java
import java.util.ArrayList;

class Vehicle {
    protected String registrationNumber;
    protected int milesTraveled;
    protected int lastMaintenanceMile;

    public Vehicle(String regNum) {
        this.registrationNumber = regNum;
        this.milesTraveled = 0;
        this.lastMaintenanceMile = 0;
    }

    public void addMiles(int miles) {
        this.milesTraveled += miles;
    }

    public void performMaintenance() {
        this.lastMaintenanceMile = this.milesTraveled;
        System.out.println("Maintenance performed for vehicle: " + this.registrationNumber);
    }

    public void checkMaintenanceDue() {
        if ((this.milesTraveled - this.lastMaintenanceMile) > 10000) {
            System.out.println("Vehicle: " + this.registrationNumber + " needs maintenance!");
        } else {
            System.out.println("No maintenance required for vehicle: " + this.registrationNumber);
        }
    }

    public void displayDetails() {
        // Implemented in s

## Summary

To conclude we learnt that invoking the LLM without any context might not yield the desired results. By adding context and further using the the prompt template to constrain the output from the LLM we are able to successfully get our desired output